In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("theappname").getOrCreate()
df = spark.read.csv('/Code/spark/LAB4/data/20*.csv', mode="DROPMALFORMED", inferSchema=True, header = True)


from pyspark.sql.functions import  length, lpad
from pyspark.sql.types import IntegerType
selection = df\
    .where((df.Diverted == 0) & (df.CancellationCode.isNull()) & (df.TailNum.rlike("^([A-Z]|[a-z]|[0-9])"
                                                                                                                  "+$")) & ((length(df.DepTime) == 4) | (length(df.DepTime) == 3)) & (length(df.ArrTime) == 4) | (length(df.ArrTime) == 3))\
    .where(df['TailNum'].isNotNull())\
    .withColumn("DepTime", when(length(df.DepTime) == 3, lpad(df['DepTime'], 4,'0')).otherwise(df['DepTime']))\
    .withColumn("ArrTime", when(length(df.ArrTime) == 3, lpad(df['ArrTime'], 4,'0')).otherwise(df['ArrTime']))\
    .sort(asc('Year'), 'Month', 'DayofMonth', 'ArrTime')\
    .withColumn("ArrDelay", df.ArrDelay.cast(IntegerType()))\
    .withColumn("DepDelay", df["DepDelay"].cast(IntegerType()))\
    .withColumn("CarrierDelay", df["CarrierDelay"].cast(IntegerType()))\
    .withColumn("WeatherDelay", df["WeatherDelay"].cast(IntegerType()))\
    .withColumn("NASDelay", df["NASDelay"].cast(IntegerType()))\
    .withColumn("SecurityDelay", df["SecurityDelay"].cast(IntegerType()))\
    .withColumn("LateAircraftDelay", df["LateAircraftDelay"].cast(IntegerType()))\
    .fillna(0, subset=['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay' ])

In [2]:
from pyspark.sql.functions import lag
from pyspark.sql.types import BinaryType
from pyspark.sql import Window
#w = Window.orderBy("Year", "Month", "DayofMonth", "ArrTime")
w = Window.partitionBy ("Year", "Month", "DayofMonth", "ArrTime").orderBy("Year", "Month", "DayofMonth", "ArrTime")

udf(returnType=BinaryType())
IsLate  = lambda x : x["ArrDelay"] > 15

base = selection \
    .withColumn("NbrOfPreviousLateFlights",(lag(selection['ArrDelay'], 1,0).over(w)  > 15 if 1 else 0).cast(IntegerType()) + (lag(selection['ArrDelay'], 2,0).over(w)  > 15 if 1 else 0).cast(IntegerType()) + (lag(selection['ArrDelay'], 3,0).over(w)  > 15 if 1 else 0).cast(IntegerType()))\
    .withColumn("IsLate", (IsLate(selection)))

In [53]:
selection.filter(selection['TailNum'].isNull()).show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------

In [10]:
spark.catalog.clearCache()

In [3]:
superBasePanda = base.toPandas()
from pandas import notnull, isnull
basePanda = superBasePanda.where(superBasePanda['TailNum'].notnull())

In [62]:
basePanda.where(basePanda['TailNum'].isnull()).count()

Year                 0
Month                0
DayofMonth           0
DayOfWeek            0
DepTime              0
CRSDepTime           0
ArrTime              0
CRSArrTime           0
UniqueCarrier        0
FlightNum            0
TailNum              0
ActualElapsedTime    0
CRSElapsedTime       0
AirTime              0
ArrDelay             0
DepDelay             0
Origin               0
Dest                 0
Distance             0
TaxiIn               0
TaxiOut              0
Cancelled            0
CancellationCode     0
Diverted             0
CarrierDelay         0
WeatherDelay         0
NASDelay             0
SecurityDelay        0
LateAircraftDelay    0
dtype: int64

In [ ]:
basePanda.head()

In [8]:
basePanda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13939481 entries, 0 to 13939480
Data columns (total 31 columns):
Year                        int32
Month                       int32
DayofMonth                  int32
DayOfWeek                   int32
DepTime                     object
CRSDepTime                  int32
ArrTime                     object
CRSArrTime                  int32
FlightNum                   int32
ActualElapsedTime           object
CRSElapsedTime              object
AirTime                     object
ArrDelay                    int32
DepDelay                    int32
Distance                    int32
TaxiIn                      object
TaxiOut                     object
Cancelled                   int32
CancellationCode            object
Diverted                    int32
CarrierDelay                int32
WeatherDelay                int32
NASDelay                    int32
SecurityDelay               int32
LateAircraftDelay           int32
NbrOfPreviousLateFlights   

In [4]:
sample = basePanda.take([0,1,2,3,4,5])

sample.info()
sample.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 31 columns):
Year                        6 non-null int32
Month                       6 non-null int32
DayofMonth                  6 non-null int32
DayOfWeek                   6 non-null int32
DepTime                     6 non-null object
CRSDepTime                  6 non-null int32
ArrTime                     6 non-null object
CRSArrTime                  6 non-null int32
UniqueCarrier               6 non-null object
FlightNum                   6 non-null int32
TailNum                     6 non-null object
ActualElapsedTime           6 non-null object
CRSElapsedTime              6 non-null object
AirTime                     6 non-null object
ArrDelay                    6 non-null int32
DepDelay                    6 non-null int32
Origin                      6 non-null object
Dest                        6 non-null object
Distance                    6 non-null int32
TaxiIn                      6 non-n

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,NbrOfPreviousLateFlights,IsLate
0,2007,1,1,1,2108,2054,0100,44,XE,3230,...,0,None,0,14,0,2,0,0,0,True
1,2007,1,1,1,2322,2138,0100,2316,NW,1062,...,0,None,0,72,0,0,0,32,1,True
2,2007,1,1,1,2118,2030,0100,2335,AA,185,...,0,None,0,31,0,37,0,17,2,True
3,2007,1,1,1,2222,2130,0100,2330,B6,1087,...,0,None,0,0,0,44,0,46,3,True
4,2007,1,1,1,2129,2040,0101,20,AA,624,...,0,None,0,29,0,0,0,12,0,True


In [5]:
x = sample[['Year', 'Month', 'DayofMonth', 'DepTime']]
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 4 columns):
Year          6 non-null int32
Month         6 non-null int32
DayofMonth    6 non-null int32
DepTime       6 non-null object
dtypes: int32(3), object(1)
memory usage: 168.0+ bytes


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pandas import notnull;
# Split the data into training and testing sets
lb_make = LabelEncoder()
basePanda = basePanda[['Year', 'Month', 'DayofMonth', 'DepTime', 'Origin', 'Dest',  'IsLate']]
basePanda = basePanda.dropna()
basePanda['Origin_code'] = lb_make.fit_transform (basePanda['Origin'])
basePanda['Dest_code'] = lb_make.fit_transform (basePanda['Dest'])
#basePanda['UniqueCarrier_code'] = lb_make.fit_transform (basePanda['UniqueCarrier'])
#basePanda['TailNum_code'] = lb_make.fit_transform (basePanda['TailNum'])
#basePanda['ArrTime_code'] = lb_make.fit_transform (basePanda['ArrTime'])
#basePanda['DepTime_code'] = lb_make.fit_transform (basePanda['DepTime'])
basePanda = basePanda.drop('Origin', axis = 1) 
basePanda = basePanda.drop('Dest', axis = 1)
#basePanda = basePanda.drop('UniqueCarrier', axis = 1) 
#basePanda = basePanda.drop('TailNum', axis = 1) 
#basePanda = basePanda.drop('CancellationCode', axis = 1)
#basePanda = basePanda.drop('ArrTime', axis = 1)
#basePanda = basePanda.drop('DepTime', axis = 1)
labels = np.array(basePanda['IsLate'])
    

features = basePanda.drop('IsLate', axis = 1) 
#features = basePanda.get_dummies(features)
features = np.array(features)


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [17]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 8, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [20]:
from time import gmtime, strftime
 
strftime("%Y-%m-%d %H:%M:%S", gmtime())
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = np.abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.round(np.mean(errors), 2))
strftime("%Y-%m-%d %H:%M:%S", gmtime())

 #%%
 
 print(predictions)

IndentationError: unexpected indent (<ipython-input-20-fb2fd422b265>, line 14)